In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Input, SimpleRNN, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)


# Things you should automatically know and have memorized
# N = number of samples
# T = sequence length
# D = number of input features
# M = number of hidden units
# K = number of output units


# Make some data
N = 1
T = 10
D = 3
K = 2
X = np.random.randn(N, T, D)

# Make an RNN (totally random sample, the result are meaningful ma serve capire le dimensioni dei layer)
M = 5 # number of hidden units
i = Input(shape=(T, D))
x = SimpleRNN(M)(i)
x = Dense(K)(x)
#NB no output activation function -> regression (se no softmax)

model = Model(i, x)

# Get the output
Yhat = model.predict(X)
print("\n\nPrediction Y_hat: ", Yhat, "\n\n")

# See if we can replicate this output
# Get the weights first
model.summary()

# Dice che non è ben chiaro come siano memorizzati i dati nella RNN (?), in questo caso in 3 array


2.9.2


1/1 [==============================] - 0s 163ms/step


Prediction Y_hat:  [[ 0.03226585 -0.5292032 ]] 


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 10, 3)]           0         
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 5)                 45        
                                                                 
 dense_4 (Dense)             (None, 2)                 12        
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# See what are the hidden layer -> i 3 array
model.layers[1].get_weights()
# E dopo ne vediamo le shapes


[array([[ 0.5437574 ,  0.5717804 , -0.2592594 , -0.1813659 , -0.42967603],
        [-0.4329113 , -0.7363736 , -0.11026406,  0.03758347,  0.54653865],
        [-0.39470944, -0.5682249 ,  0.03112137,  0.3828748 ,  0.78273445]],
       dtype=float32),
 array([[ 0.41293657,  0.7683233 ,  0.34412372,  0.32986805,  0.10921767],
        [ 0.12821284, -0.06725136, -0.6990866 ,  0.6139556 ,  0.3367122 ],
        [-0.17545147, -0.38737962,  0.48847145,  0.7066643 , -0.2849122 ],
        [-0.8632086 ,  0.474762  , -0.12184922,  0.11463995, -0.03850234],
        [ 0.19269802,  0.1723211 , -0.37337008,  0.04157759, -0.8899657 ]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32)]

In [ ]:
# Check their shapes
# Should make sense
# First output is input > hidden
# Second output is hidden > hidden
# Third output is bias term (vector of length M)
a, b, c = model.layers[1].get_weights()
print(a.shape, b.shape, c.shape)

(3, 5) (5, 5) (5,)


In [ ]:
#Otteniamo quindi i valori manualmente e li usamo per costruire noi la RNN
Wx, Wh, bh = model.layers[1].get_weights()
Wo, bo = model.layers[2].get_weights()

h_last = np.zeros(M) # initial hidden state
x = X[0] # the one and only sample
Yhats = [] # where we store the outputs

for t in range(T):
  h = np.tanh(x[t].dot(Wx) + h_last.dot(Wh) + bh)
  y = h.dot(Wo) + bo # we only care about this value on the last iteration
  Yhats.append(y)
  
  # important: assign h to h_last
  h_last = h

# print the final output
print("Y_hat: ", Yhats[-1])
print("Y_hat Deve essere uguale a quello stampato precedentemente ottenuto con model.predict")

Y_hat:  [ 0.03226589 -0.52920322]
Y_hat Deve essere uguale a quello stampato precedentemente ottenuto con model.predict
